*감성사전 키워드 추출*
-> keyword(감성사전)

*fasttext 키워드 추출*
-> keyword(fasttext_1_gram), keyword(fasttext_2_grams), keyword(fasttext_n_grams)

*fasttext 인코딩 오류 제거*
-> keyword(fasttext_1_gram_오류제거), keyword(fasttext_2_grams_오류제거), keyword(fasttext_n_grams_오류제거)

*키워드 병합*
-> keyword(3ver_결측값제거)

*one len keyword 처리*
-> keyword(4ver_결측값제거), keyword(final)

keyword(final): 타원 생성에 사용한 최종 데이터

## **1. 감성사전에서 키워드 추출**

In [ ]:
!pip install konlpy

In [ ]:
import json
import pandas as pd
import re
from konlpy.tag import Okt

# KNU 한국어 감정 사전 로드
with open('원본 데이터/SentiWord_info.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# CSV 파일 읽기
korean_data = pd.read_csv('원본 데이터/korean_data.csv')

# 단어, 감정 강도 리스트
senti_dict = {entry['word']: abs(float(entry['polarity'])) for entry in data}

# df 형태로 변환
senti_df = pd.DataFrame.from_dict(data=senti_dict, orient='index')
senti_df.reset_index(inplace=True)
senti_df.rename(columns={'index':'word', 0:'polarity'}, inplace=True)

# 특수문자 제거
cleaned_korean_data = []

for i in range(len(korean_data)):
  input_string = korean_data['Text'][i]
  cleaned_string = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", input_string)
  cleaned_korean_data.append(cleaned_string)

# KoNLPy 형태소 분석기 초기화
okt = Okt()

### 감정 스코어 계산 함수 ###
def calculate_emotion_score(word):
    score = 0.0
    if word in senti_dict:
        score = senti_dict[word]
    return score

### n-gram 생성 ###
def generate_ngrams(sentence, n):
    """
    주어진 문장을 n-gram으로 나누어서 리스트로 반환합니다.

    :param sentence: 나눌 문장
    :param n: n-gram의 n 값 (1부터 n까지의 n-gram을 생성)
    :return: n-gram을 저장한 리스트
    """
    ngrams = []

    # 문장을 공백을 기준으로 단어로 분리
    words = sentence.split()

    # n-gram 생성
    for i in range(len(words) - n + 1):
        ngram = ' '.join(words[i:i + n])
        ngrams.append(ngram)

    return ngrams

### 각 요소의 감정 스코어 계산 ###
def extract_emotional_keywords(sentence, num_keywords=5):

    # 1-gram부터 n-gram까지 생성
    words = []
    words = words + okt.morphs(sentence)

    for n in range(1, sentence.count(' ')+2):
      ngrams = generate_ngrams(sentence, n)
      words.extend(ngrams)

    # 각 단어의 감정 스코어 계산
    emotion_scores = [(word, calculate_emotion_score(word)) for word in words]

    # 스코어가 높은 순으로 정렬
    emotion_scores.sort(key=lambda x: x[1], reverse=True)

    if not emotion_scores:
      keywords = ""
      return keywords

    # 상위 num_keywords 개의 감정 키워드 추출
    keywords = [word for word, _ in emotion_scores[:num_keywords]]

    return keywords

### n-gram이 사전안에 있다면, keyword 추출 ###
korean_data['emotion_keyword'] = ""

for i in range(len(korean_data)):
  sentence = cleaned_korean_data[i]

  # 감정 키워드 추출
  emotion_keywords = extract_emotional_keywords(sentence)

  if emotion_keywords:
      #print("감정 키워드를 찾았습니다:")
      #print(emotion_keywords)
      korean_data['emotion_keyword'][i] = emotion_keywords[0]
  #else:
      #print("감정 키워드를 찾지 못했습니다.")

In [ ]:
korean_data

In [ ]:
# korean_data.to_csv('키워드 데이터/keyword(감성사전).csv', index=False)

## **2. fasttext 사용하여 키워드 추출**

cc.ko.300.bin.gz는 해당 fasttext 사이트에서 Korean bin 파일 다운 받아 사용

https://fasttext.cc/docs/en/crawl-vectors.html

In [ ]:
!pip install gensim

In [ ]:
!gunzip cc.ko.300.bin.gz

In [ ]:
!pip install konlpy

In [ ]:
import pandas as pd
import re
from konlpy.tag import Okt
from gensim import models

# CSV 파일 읽기
korean_data = pd.read_csv('원본 데이터/korean_data.csv')

ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin', encoding='utf-8')

# 특수문자 제거
cleaned_korean_data = []

for i in range(len(korean_data)):
  input_string = korean_data['Text'][i]
  cleaned_string = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", input_string)
  cleaned_korean_data.append(cleaned_string)

# KoNLPy 형태소 분석기 초기화
okt = Okt()

### n-gram 생성 (1-gram: n = 1, 2-grams: n = 2) ###
def generate_ngrams(sentence, n):
    """
    주어진 문장을 n-gram으로 나누어서 리스트로 반환합니다.

    :param sentence: 나눌 문장
    :param n: n-gram의 n 값 (1부터 n까지의 n-gram을 생성)
    :return: n-gram을 저장한 리스트
    """
    ngrams = []

    # 문장을 공백을 기준으로 단어로 분리
    words = sentence.split()

    # n-gram 생성
    for i in range(len(words) - n + 1):
        ngram = ' '.join(words[i:i + n])
        ngrams.append(ngram)

    return ngrams

### 각 요소의 감정 스코어 계산 ###
def extract_emotional_keywords(sentence, target_emotion, num_keywords=5):

    # 1-gram부터 n-gram까지 생성
    words = []
    words = words + okt.morphs(sentence)

    #for n in range(1, sentence.count(' ')+2):
    for n in range(1, sentence.count(' ')+2):
      ngrams = generate_ngrams(sentence, n)
      words.extend(ngrams)

    # 가장 유사한 단어를 찾습니다.
    most_similar_word = None
    max_similarity = -1  # 유사성 점수 초기화

    for word in words:
      similarity = ko_model.wv.similarity(target_emotion, word)
      if similarity > max_similarity:
        max_similarity = similarity
        most_similar_word = word

    return most_similar_word

### keyword 추출 ###
korean_data['emotion_keyword'] = ""

for i in range(len(korean_data)):
  sentence = cleaned_korean_data[i]
  target_emotion = korean_data['Emotion'][i]

  # 감정 키워드 추출
  most_similar_word = extract_emotional_keywords(sentence, target_emotion)

  if most_similar_word:
      #print("감정 키워드를 찾았습니다:")
      #print(emotion_keywords)
      korean_data['emotion_keyword'][i] = most_similar_word
  #else:
      #print("감정 키워드를 찾지 못했습니다.")

In [ ]:
korean_data

In [ ]:
# korean_data.to_csv('키워드 데이터/keyword(fasttext_1_gram).csv', index=False)
# korean_data.to_csv('키워드 데이터/keyword(fasttext_2_grams).csv', index=False)
# korean_data.to_csv('키워드 데이터/keyword(fasttext_n_grams).csv', index=False)

## **3. fasttext 인코딩 오류 제거**


In [ ]:
import pandas as pd

# 키워드 데이터 전부 가져오기
gram1_data = pd.read_csv('키워드 데이터/keyword(fasttext_1_gram).csv')
gram2_data = pd.read_csv('키워드 데이터/keyword(fasttext_2_grams).csv')
gramn_data = pd.read_csv('키워드 데이터/keyword(fasttext_n_grams).csv')
dict_data = pd.read_csv('키워드 데이터/keyword(감성사전).csv')

# gram1_data.isnull().sum()
# gram2_data.isnull().sum()
# gramn_data.isnull().sum()
# dict_data.isnull().sum()

# gram1_data[gram1_data['emotion_keyword'].isnull()]

# str 형태로 변경
gram1_data['emotion_keyword'] = gram1_data['emotion_keyword'].astype(str)
gram2_data['emotion_keyword'] = gram2_data['emotion_keyword'].astype(str)
gramn_data['emotion_keyword'] = gramn_data['emotion_keyword'].astype(str)
dict_data['emotion_keyword'] = dict_data['emotion_keyword'].astype(str)

# gram1_data[gram1_data['emotion_keyword'] == "\xa0"]['Emotion'].value_counts()
# gram2_data[gram2_data['emotion_keyword'] == "\xa0"]['Emotion'].value_counts()
# gramn_data[gramn_data['emotion_keyword'] == "\xa0"]['Emotion'].value_counts()

# gram1_data[gram1_data['emotion_keyword'] == "\xa0"]
# gram2_data[gram2_data['emotion_keyword'] == "\xa0"]


for i in range(len(gram1_data)):
  if gram1_data['emotion_keyword'][i] == "\xa0":
    gram1_data['emotion_keyword'][i] = gram2_data['emotion_keyword'][i]

# gram1_data[gram1_data['emotion_keyword'] == "\xa0"]

for i in range(len(gram1_data)):
  if gram1_data['emotion_keyword'][i] == "\xa0":
    gram1_data['emotion_keyword'][i] = dict_data['emotion_keyword'][i]

for i in range(len(gram2_data)):
  if gram2_data['emotion_keyword'][i] == "\xa0":
    gram2_data['emotion_keyword'][i] = dict_data['emotion_keyword'][i]

for i in range(len(gramn_data)):
  if gramn_data['emotion_keyword'][i] == "\xa0":
    gramn_data['emotion_keyword'][i] = dict_data['emotion_keyword'][i]

# gramn_data[gramn_data['emotion_keyword'] == "\xa0"]

In [ ]:
# gram1_data.to_csv("키워드 데이터/keyword(fasttext_1_gram_오류제거).csv", index = False)
# gram2_data.to_csv("키워드 데이터/keyword(fasttext_2_gram_오류제거).csv", index = False)
# gramn_data.to_csv("키워드 데이터/keyword(fasttext_n_grams_오류제거).csv", index = False)

##**4. 키워드 병합**

In [ ]:
import pandas as pd

gram1_data = pd.read_csv('키워드 데이터/keyword(fasttext_1_gram_오류제거).csv')
gram2_data = pd.read_csv('키워드 데이터/keyword(fasttext_2_grams_오류제거).csv')
gramn_data = pd.read_csv('키워드 데이터/keyword(fasttext_n_grams_오류제거).csv')
dict_data = pd.read_csv('키워드 데이터/keyword(감성사전).csv')

dict_data.rename(columns = {'emotion_keyword' : 'Keyword(dict)'}, inplace = True)
gram1_data.rename(columns = {'emotion_keyword' : 'Keyword(gram1)'}, inplace = True)
gram2_data.rename(columns = {'emotion_keyword' : 'Keyword(gram2)'}, inplace = True)

final_df = pd.concat([dict_data,gram1_data['Keyword(gram1)']],axis=1, join='inner')
final_df = pd.concat([final_df,gram2_data['Keyword(gram2)']],axis=1, join='inner')
final_df.dropna(inplace=True)
final_df.reset_index(drop=True)
final_df

In [ ]:
# final_df.to_csv("키워드 데이터/keyword(3ver_결측값제거).csv", index = False)

##**5. one len keyword 처리**


In [ ]:
import pandas as pd

keyword_df = pd.read_csv('키워드 데이터/keyword(3ver_결측값제거).csv')

keyword_df['Keyword(dict)'] = keyword_df['Keyword(dict)'].astype(str)
keyword_df['Keyword(gram1)'] = keyword_df['Keyword(gram1)'].astype(str)
keyword_df['Keyword(gram2)'] = keyword_df['Keyword(gram2)'].astype(str)

In [ ]:
one_len_dict = {}

for i in range(len(keyword_df)):
  if len(keyword_df['Keyword(dict)'][i]) == 1:
    one_len_dict[i] = keyword_df['Keyword(dict)'][i]

one_len_gram1 = {}

for i in range(len(keyword_df)):
  if len(keyword_df['Keyword(gram1)'][i]) == 1:
    one_len_gram1[i] = keyword_df['Keyword(gram1)'][i]

one_len_gram2 = {}

for i in range(len(keyword_df)):
  if len(keyword_df['Keyword(gram2)'][i]) == 1:
    one_len_gram2[i] = keyword_df['Keyword(gram2)'][i]

print(len(one_len_dict))
print(len(one_len_gram1))
print(len(one_len_gram2))

In [ ]:
# 추출하려는 인덱스 번호 리스트
indexes_to_extract = list(one_len_dict.keys())  # 추출하려는 인덱스 번호 목록

# 인덱스 번호에 해당하는 값을 추출하여 새로운 DataFrame 생성
keyword_one_len_dict = keyword_df[keyword_df['Keyword(dict)'].isin([one_len_dict.get(idx) for idx in indexes_to_extract])]

# 결과 출력
keyword_one_len_dict

In [ ]:
# 추출하려는 인덱스 번호 리스트
indexes_to_extract = list(one_len_gram1.keys())  # 추출하려는 인덱스 번호 목록

# 인덱스 번호에 해당하는 값을 추출하여 새로운 DataFrame 생성
keyword_one_len_gram1 = keyword_df[keyword_df['Keyword(gram1)'].isin([one_len_gram1.get(idx) for idx in indexes_to_extract])]

# 결과 출력
keyword_one_len_gram1

In [ ]:
# 추출하려는 인덱스 번호 리스트
indexes_to_extract = list(one_len_gram2.keys())  # 추출하려는 인덱스 번호 목록

# 인덱스 번호에 해당하는 값을 추출하여 새로운 DataFrame 생성
keyword_one_len_gram2 = keyword_df[keyword_df['Keyword(gram2)'].isin([one_len_gram2.get(idx) for idx in indexes_to_extract])]

# 결과 출력
keyword_one_len_gram2

In [ ]:
keyword_df['Keyword(final)'] = ""

for i in range(len(keyword_df)):
  if len(keyword_df['Keyword(dict)'][i]) == 1:
    if len(keyword_df['Keyword(gram1)'][i]) != 1:
      keyword_df['Keyword(final)'][i] = keyword_df['Keyword(gram1)'][i]
    elif len(keyword_df['Keyword(gram2)'][i]) != 1:
      keyword_df['Keyword(final)'][i] = keyword_df['Keyword(gram2)'][i]
    else:
      keyword_df['Keyword(final)'][i] = keyword_df['Keyword(dict)'][i]
  else:
    keyword_df['Keyword(final)'][i] = keyword_df['Keyword(dict)'][i]

keyword_df

In [ ]:
# keyword_df.to_csv("키워드 데이터/keyword(4ver_결측값제거).csv", index = False)

new_keyword_df = keyword_df[['Text', 'Emotion', 'Arousal', 'Valence', 'Keyword(final)']]
# new_keyword_df.to_csv("키워드 데이터/keyword(final).csv", index = False)